In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers, models

# Define directories for train and validation datasets
train_dir = 'train_dir'
test_dir = 'test_dir'

# Define data generators for train and validation datasets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Set the image size and batch size
img_size = (256, 256)
batch_size = 32

# Load and augment training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Load validation data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the pre-trained VGG19 model without the top layers (include_top=False)
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the pre-trained layers so they are not trained during fine-tuning
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of the base model
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')  # 3 output classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=10, validation_data=test_generator)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Found 445 images belonging to 3 classes.
Found 112 images belonging to 3 classes.
Epoch 1/10
14/14 [==============================] - 154s 11s/step - loss: 5.8688 - accuracy: 0.4045 - val_loss: 0.9840 - val_accuracy: 0.6250
Epoch 2/10
14/14 [==============================] - 202s 15s/step - loss: 2.2431 - accuracy: 0.4966 - val_loss: 0.7081 - val_accuracy: 0.6696
Epoch 3/10
14/14 [==============================] - 324s 24s/step - loss: 0.8979 - accuracy: 0.5820 - val_loss: 0.7022 - val_accuracy: 0.6607
Epoch 4/10
14/14 [==============================] - 315s 23s/step - loss: 0.8082 - accuracy: 0.6270 - val_loss: 0.6954 - val_accuracy: 0.6696
Epoch 5/10
14/14 [==============================] - 305s 22s/step - loss: 0.8274 - accuracy: 0.6247 - val_loss: 0.6769 - val_accuracy: 0.7679
Epoch 6/10
14/14 [==============================] - 207s 14s/step - loss: 0.7804 - accuracy: 0.6652 - val_loss: 0.5717 - val_accuracy: 0.7768
Epoch 7/10
14/14 [==============================] - 138s 10s/step 